In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

First check if the environment is kaggle notebook.

In [ ]:
#!pip download datasets
#!pip download transformers
!pip install -q transformers
!pip install 'transformers[torch]'

In [ ]:
!pip install datasets

In [ ]:
!python -c "from transformers import pipeline; print(pipeline('sentiment-analysis')('I love you'))"

In [ ]:
import os
iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')

import `Path` to use it to store path to the dataset.
> `pathlib.Path` : 
it is used to treat datapath as an object and manipulate as such. provides several functions and attributes for use.



In [ ]:
from pathlib import Path

install datasets. go [here](#another_cell) to understand why we are using dataset module.
<a id='this_cell'></a>

In [ ]:
if iskaggle:
    path = Path('../input/us-patent-phrase-to-phrase-matching')
    ! pip install -q datasets
    

In [ ]:
!ls {path}

now we go into the path using the `ls` command used in bash. we see that we have three csv files.  read [this book (python for data analysis by wes kinney)](https://wesmckinney.com/book/) to get to know how to manipulate csv using pandas in python.

In [ ]:
df = pd.read_csv(path/'train.csv')
df.describe(include='object')

In [ ]:
df.head()

In [ ]:
df['input'] = 'TEXT1: ' + df.context + '; TEXT2: ' + df.target + '; ANC1: ' + df.anchor

#### (or)

In [ ]:
df.input = 'TEXT1: ' + df.context + '; TEXT2: ' + df.target + '; ANC1: ' + df.anchor

gives the same result. the dataframe column can be accessed using dot access. this column is actually a pandas series.

In [ ]:
df.head()

## SideNote :
from bing regarding the datasets module we are importing.

<a id='another_cell'></a>
The `datasets` module in Python is a community-driven open-source library of datasets provided by the HuggingFace team ¹. It is designed to standardize end-user interfaces, versioning, and documentation while providing a lightweight front-end that behaves similarly for small datasets as for internet-scale corpora ¹. **The `datasets` library provides one-line dataloaders for many public datasets, including image datasets, audio datasets, text datasets in 467 languages and dialects, etc. ¹. It also provides efficient data pre-processing for public datasets as well as your own local datasets in CSV, JSON, text, PNG, JPEG, WAV, MP3, Parquet, etc. ¹.** 

Here is an example of how to install the `datasets` library using pip:

```python
! pip install datasets
```

Once installed, you can use the `load_dataset()` function to download and pre-process any of the major public datasets provided on the HuggingFace Datasets Hub ¹. For instance, the following code loads the SQuAD dataset:

```python
squad_dataset = load_dataset("squad")
```

I hope this helps!

Source: Conversation with Bing, 11/29/2023
(1) datasets · PyPI. https://pypi.org/project/datasets/.
(2) pydataset - a Python Dataset Library - YoungWonks. https://www.youngwonks.com/blog/pydataset-a-python-dataset-library.
(3) Are there any example data sets for Python? - Stack Overflow. https://stackoverflow.com/questions/16579407/are-there-any-example-data-sets-for-python.
(4) dataset · PyPI. https://pypi.org/project/dataset/.
(5) 7. Dataset loading utilities — scikit-learn 1.3.2 documentation. https://scikit-learn.org/stable/datasets.html.
(6) undefined. https://huggingface.co/docs/datasets/installation.
(7) undefined. https://huggingface.co/docs/datasets/quickstart.

we have already dowloaded dataset in [this cell](#this_cell) quietly (by using the `-q` command).

In [ ]:
from datasets import Dataset, DatasetDict
ds = Dataset.from_pandas(df)

We store it in a Transformer dataset to tokenize and use to do our nlp. This transformer is from huggingface.

In [ ]:
ds

We do two process so that our model can understand our data. That is:
- [Tokenisation](#tokenisation) : splitting texts into words(or tokens)
- [Numeralization](#numeralization) : converting tokens to numbers.\

so now we pick a model to do just this.

In [ ]:
model_nm = "/kaggle/working/microsoft/deberta-v3-small"

In [ ]:
os.makedirs("/kaggle/working/microsoft/deberta-v3-small", exist_ok=True)
tokz.save_pretrained = "/kaggle/working/microsoft/deberta-v3-small"
model.save_pretrained = "/kaggle/working/microsoft/deberta-v3-small"

In [ ]:
cache_dir = "/kaggle/working/deberta-cache"

# Create the cache directory if it doesn't exist
os.makedirs(cache_dir, exist_ok=True)

In [ ]:
from transformers import AutoModelForSequenceClassification,AutoTokenizer
HF_DATASETS_OFFLINE=1 
TRANSFORMERS_OFFLINE=1
tokz = AutoTokenizer.from_pretrained(model_nm, local_files_only=True, cache_dir=cache_dir)
model = AutoModelForSequenceClassification.from_pretrained(model_nm, local_files_only=True, cache_dir=cache_dir)

In [ ]:
tokz.tokenize("hello everyone this is issac working at 2 28 in the morning!")

the above code is an example of tokenization of a text.

now here's a function to tokenize our `inputs` column in the dataframe.

In [ ]:
def tokenz_func(x): return tokz(x['input'])
tok_ds = ds.map(tokenz_func, batched = True)

here mapping runs tokenzing in all rows in parrallel which is faster.

In [ ]:
tok_ds['input'][0], tok_ds['input_ids'][0]

these numbers are ids that each token have in the `vocab` of the tokenizer. we can use the `vocab` function to know the id for a string token. this `vocab` is a `list` btw.

In [ ]:
tokz.vocab['fia']


we need to rename the `score` column in the ds as `labels` so that transformers can identify it as the target column.

In [ ]:
tok_ds = tok_ds.rename_columns({'score':'labels'})

a separate **validation set** has to be create so that we an avoid *overfitting*. always have three separate sets of data : 
- training set
- test set
- validation set  

learn what is overfitting and underfitting and how to avoid them from [here](https://www.kaggle.com/code/jhoward/getting-started-with-nlp-for-absolute-beginners?scriptVersionId=95800123&cellId=63). this validation set used to check if our model is underfit, overfit or just right.  
>This is a set of data that we "hold out" from training -- we don't let our model see it at all. If you use the fastai library, it automatically creates a validation set for you if you don't have one, and will always report metrics (measurements of the accuracy of a model) using the validation set.  
The validation set is only ever used to see how we're doing. It's never used as inputs to training the model.

In [ ]:
dds = tok_ds.train_test_split(0.25, seed=42)
dds

In `transformers` we use `train_test_split` to split our validation set in which here 25% is used for validation.

Dr. Rachel Thomas explains [How (and why) to create a good validation set](https://www.fast.ai/2017/11/13/validation-sets/) here in this article. which i have yet to read but just procrastinated because my head hurts and it's 3 10 in the morning. ugh.

In [ ]:
#here's the test set in dataframe
eval_df = pd.read_csv(path/'test.csv')
eval_df.describe()

In [ ]:
#a test dataset is created now from the test.csv.
#this is actually held from training and showing metrics.
#it is only checked after all the training is done
#to find the accuracy of the model.
eval_df['input'] = 'TEXT1: ' + eval_df.context + '; TEXT2: ' + eval_df.target + '; ANC1: ' + eval_df.anchor
eval_ds = Dataset.from_pandas(eval_df).map(tokenz_func, batched=True)

Read about [The problem with metrics is a big problem for AI](https://www.fast.ai/2019/09/24/metrics/) as well later, just as our sensei jphoward suggested.

There are lots of metrics we try to maximize or minimize. kaggle competitions usually tell you the metric that they want you to maximize.  
Here [Pearson Coefficient Constant](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient) is being calculated between the predicted and actual similarity scores." 
>This coefficient is usually abbreviated using the single letter r. It is the most widely used measure of the degree of relationship between two variables.  
r can vary between -1, which means perfect inverse correlation, and +1, which means perfect positive correlation.

[Here](https://www.kaggle.com/code/jhoward/getting-started-with-nlp-for-absolute-beginners?scriptVersionId=95800123&cellId=91)'s where you can to get the intuition to it.

#### Snippets to calculate the correlation

In [ ]:
def corr(x,y): return np.corrcoef(x,y)[0][1]
def corr_d(eval_pred): return {'pearson': corr(*eval_pred)}

#### Atlast we come to training our model.

In [ ]:
from transformers import TrainingArguments,Trainer

set batch size and number of epochs.  
set learning rate.  
> The most important hyperparameter is the learning rate. fastai provides a learning rate finder to help you figure this out, but Transformers doesn't, so you'll just have to use trial and error. The idea is to find the largest value you can, but which doesn't result in training failing.

In [ ]:
bs = 128
epochs = 4
lr = 8e-5

`TrainingArguments` is used to set arguments by Transformers.

In [ ]:
args = TrainingArguments('outputs', learning_rate=lr, warmup_ratio=0.1, lr_scheduler_type='cosine', fp16=True,
    evaluation_strategy="epoch", per_device_train_batch_size=bs, per_device_eval_batch_size=bs*2,
    num_train_epochs=epochs, weight_decay=0.01, report_to='none')

In [ ]:
#bring model and data together
model = AutoModelForSequenceClassification.from_pretrained(model_nm, num_labels=1)
trainer = Trainer(model, args, train_dataset=dds['train'], eval_dataset=dds['test'],
                  tokenizer=tokz, compute_metrics=corr_d)

In [ ]:
#train the model
trainer.train();

now test them on the test set.

In [ ]:
preds = trainer.predict(eval_ds).predictions.astype(float)
preds = np.clip(preds, 0, 1) #clip them so that you don't see values above 1 and below 0.
preds

now to create a submission file for the competition.

In [ ]:
import datasets

submission = datasets.Dataset.from_dict({
    'id': eval_ds['id'],
    'score': preds
})

submission.to_csv('submission.csv', index=False)

> Unfortunately this is a code competition and internet access is disabled. That means the pip install datasets command we used above won't work if you want to submit to Kaggle. To fix this, you'll need to download the pip installers to Kaggle first, as described here. Once you've done that, disable internet in your notebook, go to the Kaggle leaderboards page, and click the Submission button.

--notes from sensei jphoward

**go do [disaster tweets](https://www.kaggle.com/datasets/vstepanenko/disaster-tweets/data) as practise.**  
**grandmaster solution to the competition by our sensei jphoward [here](https://www.kaggle.com/code/jhoward/iterate-like-a-grandmaster/)**